# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 28 2022 9:04AM,242944,19,60018286,"GUSA Granules USA, Inc.",Released
1,Jul 28 2022 8:40AM,242941,10,0085912167,ISDIN Corporation,Released
2,Jul 28 2022 8:40AM,242941,10,0085912169,ISDIN Corporation,Released
3,Jul 28 2022 8:40AM,242941,10,0085912172,ISDIN Corporation,Released
4,Jul 28 2022 8:40AM,242941,10,0085911500,ISDIN Corporation,Released
5,Jul 28 2022 8:40AM,242941,10,0085911441,ISDIN Corporation,Released
6,Jul 28 2022 8:40AM,242941,10,0085911442,ISDIN Corporation,Released
7,Jul 28 2022 8:40AM,242941,10,0085911445,ISDIN Corporation,Released
8,Jul 28 2022 8:40AM,242941,10,0085911446,ISDIN Corporation,Released
9,Jul 28 2022 8:40AM,242941,10,0085911452,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,242934,Released,10
39,242936,Released,27
40,242937,Released,11
41,242941,Released,57
42,242944,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242934,NaN,2.0,10.0
242936,NaN,NaN,27.0
242937,NaN,NaN,11.0
242941,NaN,NaN,57.0
242944,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242846,19.0,10.0,1.0
242851,0.0,0.0,1.0
242855,0.0,0.0,1.0
242856,0.0,0.0,1.0
242857,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242846,19,10,1
242851,0,0,1
242855,0,0,1
242856,0,0,1
242857,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242846,19,10,1
1,242851,0,0,1
2,242855,0,0,1
3,242856,0,0,1
4,242857,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242846,19,10,1
1,242851,,,1
2,242855,,,1
3,242856,,,1
4,242857,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 28 2022 9:04AM,242944,19,"GUSA Granules USA, Inc."
1,Jul 28 2022 8:40AM,242941,10,ISDIN Corporation
58,Jul 28 2022 8:36AM,242934,20,Alumier Labs Inc.
70,Jul 28 2022 8:30AM,242914,10,"Citieffe, Inc."
71,Jul 28 2022 8:30AM,242906,10,ISDIN Corporation
77,Jul 28 2022 8:30AM,242904,10,ISDIN Corporation
117,Jul 28 2022 8:28AM,242936,10,ISDIN Corporation
144,Jul 28 2022 8:26AM,242937,10,ISDIN Corporation
155,Jul 28 2022 8:20AM,242930,10,ISDIN Corporation
169,Jul 28 2022 8:06AM,242901,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 28 2022 9:04AM,242944,19,"GUSA Granules USA, Inc.",,,1
1,Jul 28 2022 8:40AM,242941,10,ISDIN Corporation,,,57
2,Jul 28 2022 8:36AM,242934,20,Alumier Labs Inc.,,2,10
3,Jul 28 2022 8:30AM,242914,10,"Citieffe, Inc.",,,1
4,Jul 28 2022 8:30AM,242906,10,ISDIN Corporation,,,34
5,Jul 28 2022 8:30AM,242904,10,ISDIN Corporation,,2,10
6,Jul 28 2022 8:28AM,242936,10,ISDIN Corporation,,,27
7,Jul 28 2022 8:26AM,242937,10,ISDIN Corporation,,,11
8,Jul 28 2022 8:20AM,242930,10,ISDIN Corporation,,,14
9,Jul 28 2022 8:06AM,242901,20,Alumier Labs Inc.,,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 9:04AM,242944,19,"GUSA Granules USA, Inc.",1,,
1,Jul 28 2022 8:40AM,242941,10,ISDIN Corporation,57,,
2,Jul 28 2022 8:36AM,242934,20,Alumier Labs Inc.,10,2,
3,Jul 28 2022 8:30AM,242914,10,"Citieffe, Inc.",1,,
4,Jul 28 2022 8:30AM,242906,10,ISDIN Corporation,34,,
5,Jul 28 2022 8:30AM,242904,10,ISDIN Corporation,10,2,
6,Jul 28 2022 8:28AM,242936,10,ISDIN Corporation,27,,
7,Jul 28 2022 8:26AM,242937,10,ISDIN Corporation,11,,
8,Jul 28 2022 8:20AM,242930,10,ISDIN Corporation,14,,
9,Jul 28 2022 8:06AM,242901,20,Alumier Labs Inc.,6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 9:04AM,242944,19,"GUSA Granules USA, Inc.",1,,
1,Jul 28 2022 8:40AM,242941,10,ISDIN Corporation,57,,
2,Jul 28 2022 8:36AM,242934,20,Alumier Labs Inc.,10,2,
3,Jul 28 2022 8:30AM,242914,10,"Citieffe, Inc.",1,,
4,Jul 28 2022 8:30AM,242906,10,ISDIN Corporation,34,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 9:04AM,242944,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,Jul 28 2022 8:40AM,242941,10,ISDIN Corporation,57.0,NaN,NaN
2,Jul 28 2022 8:36AM,242934,20,Alumier Labs Inc.,10.0,2.0,NaN
3,Jul 28 2022 8:30AM,242914,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Jul 28 2022 8:30AM,242906,10,ISDIN Corporation,34.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 28 2022 9:04AM,242944,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,Jul 28 2022 8:40AM,242941,10,ISDIN Corporation,57.0,0.0,0.0
2,Jul 28 2022 8:36AM,242934,20,Alumier Labs Inc.,10.0,2.0,0.0
3,Jul 28 2022 8:30AM,242914,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Jul 28 2022 8:30AM,242906,10,ISDIN Corporation,34.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3157778,183.0,19.0,25.0
16,242907,1.0,0.0,0.0
19,1457471,53.0,9.0,2.0
20,1214472,37.0,12.0,0.0
21,971426,4.0,0.0,0.0
22,242851,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3157778,183.0,19.0,25.0
1,16,242907,1.0,0.0,0.0
2,19,1457471,53.0,9.0,2.0
3,20,1214472,37.0,12.0,0.0
4,21,971426,4.0,0.0,0.0
5,22,242851,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,183.0,19.0,25.0
1,16,1.0,0.0,0.0
2,19,53.0,9.0,2.0
3,20,37.0,12.0,0.0
4,21,4.0,0.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,183.0
1,16,Released,1.0
2,19,Released,53.0
3,20,Released,37.0
4,21,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21,22
Status,,,,,,
Completed,25.0,0.0,2.0,0.0,0.0,0.0
Executing,19.0,0.0,9.0,12.0,0.0,0.0
Released,183.0,1.0,53.0,37.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21,22
0,Completed,25.0,0.0,2.0,0.0,0.0,0.0
1,Executing,19.0,0.0,9.0,12.0,0.0,0.0
2,Released,183.0,1.0,53.0,37.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21,22
0,Completed,25.0,0.0,2.0,0.0,0.0,0.0
1,Executing,19.0,0.0,9.0,12.0,0.0,0.0
2,Released,183.0,1.0,53.0,37.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()